In [ ]:
from aiida import load_profile,orm
load_profile();
import ipywidgets as widgets
from IPython.display import display
from widgets import sens_query, inversion_params
import aiidalab_widgets_base as awb
from settings import *
import yaml

out = widgets.Output()

# Inversion

### Import data

In [ ]:
style = {"description_width": "initial"}
address = widgets.Text(
            placeholder = 'Address to files',
            description="Remote address",
            style=style,
        )
def prepare_inspect():
    computer = orm.load_computer(label='daint')
    remote_path = orm.RemoteData(remote_path = address.value,
                             computer = computer)
    builder = INSPECT.get_builder()
    builder.remotes = {'a':remote_path,}
                      #'b':remote_path_2}
    return builder

btn_submit_1 = awb.SubmitButtonWidget(
    INSPECT,
    inputs_generator = prepare_inspect,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_1.btn_submit.button_style="info"
btn_submit_1.btn_submit.description="Launch import"

line_h = widgets.HTML(
            value="""<hr>
            """
        )
info_text = widgets.HTML(
            value="""Find for nc lise in stash folder of previous 
            Collect sensitivities calclations.
            """
        )

def prepare_inspect_cs():
    qb = orm.QueryBuilder()
    qb.append(COLLECT_SENS, tag="cs", 
              filters={"attributes.exit_status": 0}
              )
    qb.append(orm.RemoteStashFolderData, with_incoming="cs", project="*")
    builder = INSPECT.get_builder()
    directories_ = {f'test_{j}':i[0] for j,i in enumerate(qb.all())}
    if directories_:
        builder.remotes_cs = directories_
        return builder
    else:
        print('No files found.')

btn_submit_2 = awb.SubmitButtonWidget(
    INSPECT,
    inputs_generator = prepare_inspect_cs,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_2.btn_submit.button_style="success"
btn_submit_2.btn_submit.description="Launch CS import"

acc = widgets.Accordion(
            children=[
                 widgets.VBox(
                    children=[
                address,
                btn_submit_1,
                line_h,
                info_text,
                btn_submit_2
                    ])
            ],
        )
acc.set_title(0, "Import")
acc.selected_index = None
acc

### Step 1

In [ ]:
sq = sens_query.SearchSens()
sq

### Step 2

In [ ]:
inv = inversion_params.InversionParams()
inv


In [ ]:
inversion_button = widgets.Button(description = 'Submit Inversion',
                                  button_style = "success")
inverstion_parameters = inv.construct_dict()
def on_click(b):
    out.clear_output()
    inverstion_parameters.update({'sites':{i['name']:i for i in sq.list_info_obs}})
    with open('params_inversion.yaml','w') as f:
        _ = yaml.dump(inverstion_parameters,f)

    
inversion_button.on_click(on_click)
display(inversion_button,out)

### Step 3

In [ ]:
def prepare_inversion():
    
    inverstion_parameters = inv.construct_dict()
    inverstion_parameters.update({'sites':{i['name']:i for i in sq.list_info_obs}})

    builder = INVERSION.get_builder()
    builder.code = orm.load_code('inversion@daint')
    builder.remotes = sq.list_remotes
    builder.observations = sq.selected_obs
    builder.inv_params = orm.Dict(inverstion_parameters)
    builder.start_date = orm.Str(sq.date_range.value[:10])
    builder.end_date = orm.Str(sq.date_range.value[12:])
    builder.chunk = orm.Str(inv.chunk.value)
    builder.chunk_w = orm.Str(inv.chunk_w.value)
    return builder


In [ ]:
btn_submit_ = awb.SubmitButtonWidget(
    INVERSION,
    inputs_generator = prepare_inversion,
    disable_after_submit = False,
    append_output = True,
)
btn_submit_.btn_submit.button_style="success"
display(btn_submit_, out)